In [1]:
import re
import json
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# 1. Load Data (Adjusted for spam.txt)
# Most spam.txt files are Tab Separated. If yours uses commas, change sep='\t' to sep=','
df = pd.read_csv("spam.txt", sep='\t', names=["label", "message"], encoding="latin-1")
df["label"] = df["label"].map({"ham": 0, "spam": 1})

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["cleaned"] = df["message"].apply(clean_text)

In [3]:
# 2. Preprocessing
VOCAB_SIZE = 8000
MAX_LEN = 100
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(df["cleaned"])

In [4]:
# SAVE TOKENIZER for JS
with open('tokenizer.json', 'w') as f:
    json.dump(tokenizer.word_index, f)

X_seq = tokenizer.texts_to_sequences(df["cleaned"])
X_pad = pad_sequences(X_seq, maxlen=MAX_LEN, padding="post")

In [5]:
# 3. Model
model = Sequential([
    Embedding(VOCAB_SIZE, 16, input_length=MAX_LEN), # Smaller embedding for light web use
    GlobalAveragePooling1D(),
    Dense(16, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_pad, df["label"], epochs=10, batch_size=32)



Epoch 1/10


175/175 [==============================] - 4s 7ms/step - loss: 0.4805 - accuracy: 0.8580
Epoch 2/10
175/175 [==============================] - 1s 7ms/step - loss: 0.3483 - accuracy: 0.8659
Epoch 3/10
175/175 [==============================] - 1s 7ms/step - loss: 0.2928 - accuracy: 0.8663
Epoch 4/10
175/175 [==============================] - 1s 7ms/step - loss: 0.1781 - accuracy: 0.9282
Epoch 5/10
175/175 [==============================] - 1s 7ms/step - loss: 0.0951 - accuracy: 0.9733
Epoch 6/10
175/175 [==============================] - 1s 6ms/step - loss: 0.0638 - accuracy: 0.9824
Epoch 7/10
175/175 [==============================] - 1s 6ms/step - loss: 0.0490 - accuracy: 0.9862
Epoch 8/10
175/175 [==============================] - 1s 6ms/step - loss: 0.0396 - accuracy: 0.9887
Epoch 9/10
175/175 [==============================] - 1s 6ms/step - loss: 0.0337 - accuracy: 0.9898
Epoch 10/10
175/175 [==============================] - 1s 7ms/step - loss: 0.0292 - accuracy: 0.

In [6]:
# 4. Save Model
model.save("spam_model.h5")
print("Model and Tokenizer saved!")

Model and Tokenizer saved!


d:\Learning101\ml_model_in_tensorflow\tfjs-env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
